In [69]:
# setup cell for realistic compositions
# safe to ignore for xray purposes
import numpy as np
from pycalphad.eq.utils import point_sample
import pycalphad.variables as v
variables = [v.SiteFraction('TEST', 0, 'AL'),
             v.SiteFraction('TEST', 0, 'NI'),
             v.SiteFraction('TEST', 0, 'CR'),
             v.SiteFraction('TEST', 0, 'FE'),
             v.SiteFraction('TEST', 1, 'AL'),
             v.SiteFraction('TEST', 1, 'NI'),
             v.SiteFraction('TEST', 1, 'CR'),
             v.SiteFraction('TEST', 1, 'FE'),
             v.SiteFraction('TEST', 2, 'VA')
             ]
constitution = point_sample([4, 4, 1], pdof=100)
composition = np.empty((constitution.shape[0], 4))

site_ratios = [0.75, 0.25, 1]
for idx, comp in enumerate(['AL', 'NI', 'CR', 'FE']):
    avector = [float(vxx.species == comp) * site_ratios[vxx.sublattice_index] for vxx in variables]
    composition[:, idx] = np.dot(constitution, avector)

In [70]:
import xray
print('constitution.shape', constitution.shape)
print('composition.shape ', composition.shape)

constitution.shape (600, 9)
composition.shape  (600, 4)


In [71]:
# Can we use a multi-index here?
# 'AL_0' is species 'AL' in sublattice 0, 'CR_1' is species 'CR' in sublattice 1, etc.
internal_dof = ['AL_0', 'NI_0', 'CR_0', 'FE_0', 'AL_1', 'NI_1', 'CR_1', 'FE_1', 'VA_2']
components = ['AL', 'NI', 'CR', 'FE']
temperatures = np.linspace(300., 1200., num=20)
pressures = np.logspace(5, 10, num=20)
site_ratios = np.array([3, 1, 1])
energies = -1e4 * np.random.rand(20, 20, 600)
ds = xray.Dataset({'energies': (['T', 'P', 'id'], energies), \
                   'constitution': (['id', 'internal_dof'], constitution), \
                   'components': components, \
                   'internal_dof': internal_dof}, \
                  coords={'Phase':'FCC_A1', 'T':temperatures, 'P':pressures, \
                          'composition': (['id', 'components'], composition), \
                          })

In [72]:
ds

<xray.Dataset>
Dimensions:       (P: 20, T: 20, components: 4, id: 600, internal_dof: 9)
Coordinates:
  * components    (components) <U2 'AL' 'NI' 'CR' 'FE'
  * internal_dof  (internal_dof) <U4 'AL_0' 'NI_0' 'CR_0' 'FE_0' 'AL_1' ...
    composition   (id, components) float64 0.153 0.2138 0.2917 0.3415 0.316 ...
    Phase         <U6 'FCC_A1'
  * P             (P) float64 1e+05 1.833e+05 3.36e+05 6.158e+05 1.129e+06 ...
  * T             (T) float64 300.0 347.4 394.7 442.1 489.5 536.8 584.2 ...
  * id            (id) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Data variables:
    constitution  (id, internal_dof) float64 0.1296 0.2055 0.301 0.3639 ...
    energies      (T, P, id) float64 -5.533e+03 -605.8 -2.507e+03 -8.546e+03 ...

In [121]:
# This is not a good way to implement nearest-neighbor search
target_composition = np.array([0.2, 0.3, 0.1, 0.4])
target_id = (ds.composition - target_composition).reduce(np.linalg.norm, 'components').argmin()
ds.energies.sel(T=1200, P=1e6, id=target_id, method='nearest')

<xray.DataArray 'energies' ()>
array(-891.1683215770149)
Coordinates:
    Phase    <U6 'FCC_A1'
    P        float64 1.129e+06
    id       int64 587
    T        float64 1.2e+03

In [122]:
ds.constitution.sel(id=target_id).to_dataframe()

,constitution,Phase,id
internal_dof,,,
AL_0,0.208199,FCC_A1,587
NI_0,0.210995,FCC_A1,587
CR_0,0.043723,FCC_A1,587
FE_0,0.537083,FCC_A1,587
AL_1,0.252339,FCC_A1,587
NI_1,0.526920,FCC_A1,587
CR_1,0.211842,FCC_A1,587
FE_1,0.008899,FCC_A1,587
VA_2,1.000000,FCC_A1,587


In [120]:
print('Dataset size:', ds.nbytes / 1e6, 'Mb')
print('DataFrame size:', ds.to_dataframe().values.nbytes / 1e6, 'Mb')

Dataset size: 1.98772 Mb
DataFrame size: 276.48 Mb
